In [ ]:
!pip install arabert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 12.6 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186455 sha256=697ad3bcac4b41c82759dfe000c9dcb7fad06199bd1eb49135a07d83f5e8358b
  Stored in directory: /root/.cache/pip/wheels/10/f0/fd/4813b1177405693e8da9cdea839f0fb64fde161380e058c827
Successfully built emoji


**1. L'importation des bibliothèques**

In [ ]:
import pandas as pd
from transformers import GPT2TokenizerFast, GPT2ForSequenceClassification, Trainer, TrainingArguments
from arabert.aragpt2.grover.modeling_gpt2 import GPT2ForSequenceClassification as ArGPT2ForSequenceClassification
from arabert.preprocess import ArabertPreprocessor
from torch.utils.data import Dataset

**2. Chargement des données**

In [ ]:
data = pd.read_excel('/content/hotel_reviews_preprocessed.xlsx')
print(data.head())

   no Hotel name  rating    user type                   room type  \
0   2    فندق 72       2  مسافر منفرد  غرفة ديلوكس مزدوجة أو توأم   
1   3    فندق 72       5          زوج  غرفة ديلوكس مزدوجة أو توأم   
2  16    فندق 72       5          زوج                           -   
3  20    فندق 72       1          زوج          غرفة قياسية مزدوجة   
4  23    فندق 72       4          زوج  غرفة ديلوكس مزدوجة أو توأم   

            nights                                  Normalized_Review  \
0  أقمت ليلة واحدة                      ممتاز النظافة والطاقم متعاون    
1  أقمت ليلة واحدة   استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ   
2      أقمت ليلتين  استثنائي انصح بأختيار الاسويت و بالاخص غرفه رق...   
3  أقمت ليلة واحدة   استغرب تقييم الفندق كخمس نجوم لا شي يستحق  نجمه    
4      أقمت ليلتين  جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان ...   

  Sentiment  
0  Negative  
1  Positive  
2  Positive  
3  Negative  
4  Positive  


**3. Préparation des données pour le fine-tuning**

In [ ]:
X = data['Normalized_Review']
y = data['Sentiment']

# Mapper les sentiments (Positive -> 1, Negative -> 0)
y = y.map({'Positive': 1, 'Negative': 0})

**4. Chargement du modèle pré-entraîné AraGPT2**

In [ ]:
MODEL_NAME = 'aubmindlab/aragpt2-base'
arabert_prep = ArabertPreprocessor(model_name=MODEL_NAME)
model = ArGPT2ForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)

# Assigner un token de padding
tokenizer.pad_token = tokenizer.eos_token

# Définir pad_token_id dans la configuration du modèle
model.config.pad_token_id = tokenizer.pad_token_id


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at aubmindlab/aragpt2-base and are newly initialized: ['emb_norm.bias', 'emb_norm.weight', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

**5. Crétion d'une classe personnalisée pour le dataset**

In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)


**6.Préparation les données d'entrée pour l'entraînement**

In [ ]:
# Tokeniser les données
encodings = tokenizer(list(X), truncation=True, padding=True, max_length=256)
train_dataset = TextDataset(encodings, list(y))

**7. Configuration des arguments d'entraînement**

In [ ]:
training_args = TrainingArguments(
    output_dir="./aragpt2-finetuned",
    num_train_epochs=10,
    # Le taux d'apprentissage
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    # Évaluer après chaque époque
    evaluation_strategy="epoch",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


**8. Création l'objet Trainer**

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    tokenizer=tokenizer
)

<ipython-input-9-7330a4a4916d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


**9. Lancement du fine-tuning**

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.461500,0.334627
2,0.400100,0.462076
3,0.337000,0.433485
4,0.401300,0.355890
5,0.189600,0.301717
6,0.419000,0.254010
7,0.282600,0.247851


Epoch,Training Loss,Validation Loss
1,0.461500,0.334627
2,0.400100,0.462076
3,0.337000,0.433485
4,0.401300,0.355890
5,0.189600,0.301717
6,0.419000,0.254010
7,0.282600,0.247851
8,0.306700,0.256640
9,0.177300,0.227207
10,0.190700,0.219008


TrainOutput(global_step=6250, training_loss=0.33910758153915405, metrics={'train_runtime': 6668.2047, 'train_samples_per_second': 14.995, 'train_steps_per_second': 0.937, 'total_flos': 1.306353104584704e+16, 'train_loss': 0.33910758153915405, 'epoch': 10.0})

**La sauvegarde du Model**

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

drive_model_dir = "/content/drive/My Drive/aragpt2-finetuned"

model.save_pretrained(drive_model_dir)
tokenizer.save_pretrained(drive_model_dir)

print(f"Le modèle est sauvegardé dans : {drive_model_dir}")


Mounted at /content/drive
Le modèle est sauvegardé dans : /content/drive/My Drive/aragpt2-finetuned


**10. Évaluation du modèle fine-tuné**

In [ ]:
from sklearn.metrics import classification_report
import torch

test_data = pd.read_excel('/content/hotel_reviews_test.xlsx')
X_test = test_data['Normalized_Review']
y_test = test_data['Sentiment'].map({'Positive': 1, 'Negative': 0})

encodings_test = tokenizer(list(X_test), truncation=True, padding=True, max_length=256)
test_dataset = TextDataset(encodings_test, list(y_test))

# Faire des prédictions sur le jeu de test
predictions = trainer.predict(test_dataset)

predicted_labels = predictions.predictions.argmax(axis=-1)  # Indices des classes prédites

# Afficher le classification_report
print(classification_report(y_test, predicted_labels, target_names=["Negative", "Positive"]))


              precision    recall  f1-score   support

    Negative       0.95      0.94      0.94      1190
    Positive       0.91      0.93      0.92       809

    accuracy                           0.93      1999
   macro avg       0.93      0.93      0.93      1999
weighted avg       0.93      0.93      0.93      1999



**11. Test du modele**

In [ ]:
from transformers import AutoTokenizer, GPT2ForSequenceClassification

model_name = "/content/drive/MyDrive/aragpt2-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name)
print("Le modèle et le tokenizer ont été chargés avec succès !")


Some weights of the model checkpoint at /content/drive/MyDrive/aragpt2-finetuned were not used when initializing GPT2ForSequenceClassification: ['transformer.emb_norm.bias', 'transformer.emb_norm.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/aragpt2-finetuned and are newly initialized: ['transformer.ln_f.bias', 'transformer.ln_f.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for p

Le modèle et le tokenizer ont été chargés avec succès !


In [ ]:
import torch

def predict_sentiment(review):
    # Prétraiter le texte
    new_review_cleaned = arabert_prep.preprocess(review)
    inputs = tokenizer(new_review_cleaned, return_tensors="pt", padding=True, truncation=True, max_length=256)

    # Faire une prédiction
    with torch.no_grad():
        model.eval()
        outputs = model(**inputs)

    # Calculer la prédiction
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

    sentiment = "Positive" if predicted_class == 1 else "Negative"
    return sentiment

while True:
    new_review = input("Entrez un avis (ou tapez 'quit' pour quitter) : ")

    if new_review.lower() == 'quit':
        print("Au revoir!")
        break

    sentiment = predict_sentiment(new_review)

    print(f"Sentiment prédit : {sentiment}")

Entrez un avis (ou tapez 'quit' pour quitter) : استمتعت بكل لحظة في هذا الفندق. الإفطار كان لذيذًا والخدمات ممتازة. سأعود بالتأكيد
Sentiment prédit : Positive
Entrez un avis (ou tapez 'quit' pour quitter) : الفندق قديم، يحتاج إلى تجديد
Sentiment prédit : Negative
Entrez un avis (ou tapez 'quit' pour quitter) : موقع الفندق ممتاز، قريب من كل شيء
Sentiment prédit : Positive
Entrez un avis (ou tapez 'quit' pour quitter) : quit
Au revoir!
